Shampoo example from data science mistery

3D data with one feature and one sequence

In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import matplotlib
# be able to save images on server
matplotlib.use('Agg')
from matplotlib import pyplot
import numpy

Using TensorFlow backend.


In [2]:
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
 
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df = df.drop(0)
	return df
 
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)
 
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, yhat):
	new_row = [x for x in X] + [yhat]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]
 
# evaluate the model on a dataset, returns RMSE in transformed units
def evaluate(model, raw_data, scaled_dataset, scaler, offset, batch_size):
	# separate
	X, y = scaled_dataset[:,0:-1], scaled_dataset[:,-1]
	# reshape
	reshaped = X.reshape(len(X), 1, 1)
	# forecast dataset
	output = model.predict(reshaped, batch_size=batch_size)
	# invert data transforms on forecast
	predictions = list()
	for i in range(len(output)):
		yhat = output[i,0]
		# invert scaling
		yhat = invert_scale(scaler, X[i], yhat)
		# invert differencing
		yhat = yhat + raw_data[i]
		# store forecast
		predictions.append(yhat)
	# report performance
	rmse = sqrt(mean_squared_error(raw_data[1:], predictions))
	return rmse
 
# fit an LSTM network to training data
def fit_lstm(train, test, raw, scaler, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])   #change it to 3D data
	# prepare model
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	# fit model
	train_rmse, test_rmse = list(), list()
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
		# evaluate model on train data
		raw_train = raw[-(len(train)+len(test)+1):-len(test)]
		train_rmse.append(evaluate(model, raw_train, train, scaler, 0, batch_size))
		model.reset_states()
		# evaluate model on test data
		raw_test = raw[-(len(test)+1):]
		test_rmse.append(evaluate(model, raw_test, test, scaler, 0, batch_size))
		model.reset_states()
	history = DataFrame()
	history['train'], history['test'] = train_rmse, test_rmse
	return history

In [4]:
# load dataset
series = read_csv('shapoo.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
	# transform data to be stationary

In [5]:
raw_values = series.values
diff_values = difference(raw_values, 1)
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

In [8]:
diff_values

0    -120.1
1      37.2
2     -63.8
3      61.0
4     -11.8
5      63.3
6      -7.3
7     -31.7
8     -69.9
9     213.6
10   -150.6
11      8.4
12    -44.8
13     60.6
14     63.2
15    -81.9
16     95.6
17    -61.0
18     77.6
19    -13.7
20    131.7
21   -157.1
22     77.8
23     -2.6
24    100.7
25   -124.5
26    123.4
27    -38.0
28     36.1
29    138.1
30   -167.9
31    274.4
32   -206.7
33    106.0
34     65.6
dtype: float64

In [10]:
supervised_values

array([[-120.1,   37.2],
       [  37.2,  -63.8],
       [ -63.8,   61. ],
       [  61. ,  -11.8],
       [ -11.8,   63.3],
       [  63.3,   -7.3],
       [  -7.3,  -31.7],
       [ -31.7,  -69.9],
       [ -69.9,  213.6],
       [ 213.6, -150.6],
       [-150.6,    8.4],
       [   8.4,  -44.8],
       [ -44.8,   60.6],
       [  60.6,   63.2],
       [  63.2,  -81.9],
       [ -81.9,   95.6],
       [  95.6,  -61. ],
       [ -61. ,   77.6],
       [  77.6,  -13.7],
       [ -13.7,  131.7],
       [ 131.7, -157.1],
       [-157.1,   77.8],
       [  77.8,   -2.6],
       [  -2.6,  100.7],
       [ 100.7, -124.5],
       [-124.5,  123.4],
       [ 123.4,  -38. ],
       [ -38. ,   36.1],
       [  36.1,  138.1],
       [ 138.1, -167.9],
       [-167.9,  274.4],
       [ 274.4, -206.7],
       [-206.7,  106. ],
       [ 106. ,   65.6]])

In [11]:
# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]
	# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)
	# fit and evaluate model
train_trimmed = train_scaled[2:, :]

In [12]:
train_trimmed

array([[-0.496628  ,  0.17669274],
       [ 0.17669274, -0.21607769],
       [-0.21607769,  0.1891017 ],
       [ 0.1891017 , -0.1917993 ],
       [-0.1917993 , -0.32344214],
       [-0.32344214, -0.52953871],
       [-0.52953871,  1.        ],
       [ 1.        , -0.96493121],
       [-0.96493121, -0.10709469],
       [-0.10709469, -0.39411923],
       [-0.39411923,  0.17453466],
       [ 0.17453466,  0.18856218],
       [ 0.18856218, -0.59428109],
       [-0.59428109,  0.3633666 ],
       [ 0.3633666 , -0.48152145],
       [-0.48152145,  0.26625303],
       [ 0.26625303, -0.22632857],
       [-0.22632857,  0.55813326],
       [ 0.55813326, -1.        ],
       [-1.        ,  0.26733207]])

In [16]:
X=train_trimmed[:,0:-1]

In [17]:
X = X.reshape(X.shape[0], 1, X.shape[1])

3D data with one feature and one sequence

In [18]:
X

array([[[-0.496628  ]],

       [[ 0.17669274]],

       [[-0.21607769]],

       [[ 0.1891017 ]],

       [[-0.1917993 ]],

       [[-0.32344214]],

       [[-0.52953871]],

       [[ 1.        ]],

       [[-0.96493121]],

       [[-0.10709469]],

       [[-0.39411923]],

       [[ 0.17453466]],

       [[ 0.18856218]],

       [[-0.59428109]],

       [[ 0.3633666 ]],

       [[-0.48152145]],

       [[ 0.26625303]],

       [[-0.22632857]],

       [[ 0.55813326]],

       [[-1.        ]]])

In [15]:
train_trimmed[:,-1]

array([ 0.17669274, -0.21607769,  0.1891017 , -0.1917993 , -0.32344214,
       -0.52953871,  1.        , -0.96493121, -0.10709469, -0.39411923,
        0.17453466,  0.18856218, -0.59428109,  0.3633666 , -0.48152145,
        0.26625303, -0.22632857,  0.55813326, -1.        ,  0.26733207])

In [19]:
# config
repeats = 10
n_batch = 4
n_epochs = 10
n_neurons = 1
	# run diagnostic tests
for i in range(repeats):
	history = fit_lstm(train_trimmed, test_scaled, raw_values, scaler, n_batch, n_epochs, n_neurons)
	pyplot.plot(history['train'], color='blue')
	pyplot.plot(history['test'], color='orange')
	print('%d) TrainRMSE=%f, TestRMSE=%f' % (i, history['train'].iloc[-1], history['test'].iloc[-1]))
pyplot.savefig('epochs_diagnostic.png')

0) TrainRMSE=91.206313, TestRMSE=135.305723
1) TrainRMSE=81.388118, TestRMSE=121.823147
2) TrainRMSE=90.118667, TestRMSE=126.393225
3) TrainRMSE=87.100322, TestRMSE=129.248256
4) TrainRMSE=85.779837, TestRMSE=124.358386
5) TrainRMSE=88.494112, TestRMSE=131.215254
6) TrainRMSE=99.981369, TestRMSE=144.089385
7) TrainRMSE=89.551811, TestRMSE=131.877691
8) TrainRMSE=87.813744, TestRMSE=129.172444
9) TrainRMSE=89.813615, TestRMSE=132.821290
